# Initialisation

In [ ]:
from __future__ import annotations

import locale
import random
from pathlib import Path

from mqt.problemsolver.qubo_tools import pathfinder as pf
from mqt.problemsolver.qubo_tools.pathfinder import Graph

In [ ]:
with Path("input/graph.txt").open(encoding=locale.getpreferredencoding(False)) as file:
    graph = Graph.read(file)

# QUBO Construction

### Parameters

We begin by setting up our parameters for the QuboGenerator. The `PathFindingQuboGenerator` supports the following parameters:

- `encoding_type`: One of `ONE_HOT`, `DOMAIN_WALL`, or `BINARY`. the encoding for the binary variables in  the QUBO formulation.
- `n_paths`: The number of paths to be searched.
- `max_path_length`: The maximum length of a path to be searched.
- `loops`: Indicates, whether the searched path should be a loop.

In [ ]:
encoding_type = pf.EncodingType.DOMAIN_WALL
n_paths = 1
max_path_length = graph.n_vertices
loops = True

settings = pf.PathFindingQuboGeneratorSettings(encoding_type, n_paths, max_path_length, loops)

Using these settings, we can now create a new `PathFindingQuboGenerator`:

In [ ]:
generator = pf.PathFindingQuboGenerator(
    objective_function=pf.MinimizePathLength(path_ids=[1]),
    graph=graph,
    settings=settings,
)

### Constraints

We can add constraints to the QUBO generator with the `add_constraint` method. For this example, we add two constraints:

1) `PathIsValid`: Enforces that the found path is actually valid (i.e. all edges in it exist).
2) `PathContainsVerticesExactlyOnce`: Enforces that the given vertices appear exactly once in the path.

In [ ]:
generator.add_constraint(pf.PathIsValid(path_ids=[1]))
generator.add_constraint(pf.PathContainsVerticesExactlyOnce(vertex_ids=graph.all_vertices, path_ids=[1]))

### Generate QUBO Formulation

There are several ways to generate the problem's QUBO formulation.

`QuboGenerator.construct()` generates a simplified mathematical expression for the problem: 

In [ ]:
generator.construct()

`QuboGenerator.construct_expansion()` generates an expanded formula of the form

$C_{1,1}x_1 + C_{1,2}x_1x_2 + C_{1,3}x_1x_3 + ... + C_{n-1,n}x_{n-1}x_n$

In [ ]:
generator.construct_expansion()

`QuboGenerator.construct_qubo_matrix()` generates the QUBO formulation as a QUBO matrix $Q$ such that the QUBO problem can be formulated as

$$\mathbf{x}^* = \argmin_\mathbf{x} \mathbf{x}^T Q \mathbf{x}$$

In [ ]:
A = generator.construct_qubo_matrix()
A

## Quantum Circuits

The constructed operator can be used to create quantum circuits that solve the optimization problem.

### QAOA

In [ ]:
seed = random.randint(10000, 20000)
qaoa = generator.construct_qaoa_circuit()
qaoa.draw(output="text", fold=-1)

# Other Features

_Use the JSON input format_

In [ ]:
with Path("input/tsp.json").open("r", encoding="utf-8") as file:
    generator_new = pf.PathFindingQuboGenerator.from_json(file.read(), graph)

A = generator_new.construct_qubo_matrix()

_Use Encoding suggestion_

The `PathFindingQuboGenerator` supports the suggestion of the optimal encoding for a given problem instance, based on the number of required binary variables. 

In [ ]:
with Path("input/tsp.json").open("r", encoding="utf-8") as file:
    print(pf.PathFindingQuboGenerator.suggest_encoding(file.read(), graph))

## Different constraints

_Also define the starting vertex of the path_

In [ ]:
generator_new = pf.PathFindingQuboGenerator(pf.MinimizePathLength([1]), graph, settings)
generator_new.add_constraint(pf.PathIsValid([1]))
generator_new.add_constraint(pf.PathContainsVerticesExactlyOnce(graph.all_vertices, [1]))


generator_new.add_constraint(pf.PathStartsAt([3], 1))


A = generator_new.construct_qubo_matrix()
print(A)

_Find the shortest paths $\pi_1$ and $\pi_2$ from $s_p$ to $t_p$ respectively that don't intersect_

In [ ]:
(s1, t1) = 1, 5
(s2, t2) = 2, 6

settings = pf.PathFindingQuboGeneratorSettings(encoding_type, n_paths=2, max_path_length=max_path_length, loops=True)
generator_new = pf.PathFindingQuboGenerator(pf.MinimizePathLength([1, 2]), graph, settings)
generator_new.add_constraint(pf.PathIsValid([1, 2]))
generator_new.add_constraint(pf.PathStartsAt([s1], 1))
generator_new.add_constraint(pf.PathStartsAt([s2], 2))
generator_new.add_constraint(pf.PathEndsAt([t1], 1))
generator_new.add_constraint(pf.PathEndsAt([t2], 2))
generator_new.add_constraint(pf.PathsShareNoVertices(1, 2))